# RANSAC-Flow: Generic Two-Stage Image Alignment
## 论文复现与原理解析

本 Notebook 旨在详细解析 ECCV 2020 论文 "RANSAC-Flow: generic two-stage image alignment"，并记录复现过程中的技术细节与踩坑指南。

### 1. 论文概述与核心贡献

**论文标题**: RANSAC-Flow: generic two-stage image alignment
**会议**: ECCV 2020
**作者**: Xi Shen, François Darmon, Alexei A. Efros, Mathieu Aubry
**链接**: [arXiv](https://arxiv.org/abs/2004.01526) | [Project Page](http://imagine.enpc.fr/~shenx/RANSAC-Flow/)

**核心问题**:
图像对齐（Image Alignment）是计算机视觉中的基础问题。传统的参数化方法（如 Homography）只能处理简单的几何变换，而基于光流（Optical Flow）的非参数化方法虽然灵活，但在大位移和显著外观变化下容易失效。

**核心贡献**:
1.  **两阶段对齐框架 (Two-Stage Alignment)**: 提出了一种由粗到精（Coarse-to-Fine）的对齐策略。
    *   **粗对齐 (Coarse)**: 使用 RANSAC 对特征匹配进行鲁棒估计，得到全局变换（如 Homography）。
    *   **精对齐 (Fine)**: 在粗对齐的基础上，使用神经网络预测残差光流（Residual Flow），处理局部形变。
2.  **无监督/自监督学习**: 该方法不需要人工标注的对应关系（Ground Truth Correspondences），可以通过重建损失（Reconstruction Loss）和循环一致性损失（Cycle Consistency Loss）进行训练。
3.  **通用性**: 该方法在多种任务上表现出色，包括艺术品对齐、3D 重建、纹理迁移等。

---

### 2. 关键技术与方法论解析

RANSAC-Flow 的核心思想是将传统的几何方法（RANSAC）与深度学习（Deep Learning）相结合。

#### 2.1 总体架构

![Overview](img/overview.jpg)
*(注：请确保 `img/overview.jpg` 存在于工作区中，否则请参考 README 中的链接)*

#### 2.2 阶段一：粗对齐 (Coarse Alignment)

这一阶段的目标是找到一个全局变换 $T$（通常是单应性矩阵 Homography），将源图像 $I_s$ 大致对齐到目标图像 $I_t$。

1.  **特征提取**: 使用预训练的 ResNet-50 提取多尺度特征。
2.  **特征匹配**: 计算源图像和目标图像特征之间的相似度矩阵，找到互为最近邻（Mutual Nearest Neighbors）的匹配点对。
3.  **RANSAC**: 使用 RANSAC 算法从匹配点对中估计最佳的单应性矩阵 $H$。
    *   RANSAC (Random Sample Consensus) 能够有效剔除错误匹配（Outliers），保证估计的鲁棒性。
4.  **变换**: 使用估计出的 $H$ 对源图像进行变换，得到粗对齐图像 $I'_s$。

#### 2.3 阶段二：精对齐 (Fine Alignment)

在粗对齐的基础上，进一步修正局部的像素级错位。

1.  **输入**: 粗对齐图像 $I'_s$ 和目标图像 $I_t$ 的特征图。
2.  **相关性计算**: 计算特征图之间的局部相关性（Local Correlation）。
3.  **光流预测**: 使用一个轻量级的全卷积网络（基于 ResNet 瓶颈层）预测残差光流 $F_{res}$。
4.  **最终变换**: 最终的变形场是粗对齐的网格变换与残差光流的叠加。

#### 2.4 损失函数

模型训练主要依赖于以下损失函数：
*   **重建损失 (Reconstruction Loss)**: 对齐后的源图像应与目标图像在外观上相似（使用 SSIM 或 L1 损失）。
*   **循环一致性损失 (Cycle Consistency Loss)**: $I_s \to I_t$ 和 $I_t \to I_s$ 的变换应该是互逆的。
*   **匹配度损失 (Matchability Loss)**: 预测哪些区域是可以匹配的，哪些是遮挡或无对应区域。

---

### 3. 核心算法实现演示

以下代码展示了如何加载模型并进行简单的推理准备。我们将复现 `quick_start/align2images.py` 中的核心逻辑。

#### 3.1 环境准备与依赖导入


In [ ]:
import sys
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# 添加项目路径以便导入模块
sys.path.append('utils/')
sys.path.append('model/')
sys.path.append('quick_start/')

# 尝试导入项目模块 (假设在项目根目录下运行)
try:
    import outil
    import model
    from coarseAlignFeatMatch import CoarseAlign
    print("模块导入成功！")
except ImportError as e:
    print(f"模块导入失败: {e}")
    print("请确保当前工作目录在 RANSAC-Flow 项目根目录下。")

# 检查 CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


#### 3.2 粗对齐 (Coarse Alignment) 核心代码解析

粗对齐依赖于 `CoarseAlign` 类。这里我们重点关注它如何使用 ResNet 提取特征并计算 Homography。

*(注：以下代码仅为演示逻辑，实际运行需要加载预训练权重)*


In [ ]:
def demo_coarse_alignment_logic():
    # 1. 初始化 CoarseAlign 模块
    # 注意：这里我们修改了源码以适配新版 torchvision (weights=...)
    # nbScale=7, coarseIter=10000, minSize=400
    coarse_model = CoarseAlign(nbScale=7, nbIter=10000, tolerance=0.05, 
                               transform='Homography', minSize=400, 
                               segId=1, segFg=True, imageNet=True, scaleR=1.2)
    
    print("CoarseAlign 模型初始化完成。")
    
    # 2. 模拟输入图像
    img1 = Image.new('RGB', (400, 300), color = 'red')
    img2 = Image.new('RGB', (400, 300), color = 'blue')
    
    # 3. 设置源图像和目标图像 (会触发特征提取)
    # coarse_model.setSource(img1)
    # coarse_model.setTarget(img2)
    
    # 4. 计算最佳变换参数 (RANSAC)
    # bestPrm, inlierMask = coarse_model.getCoarse(np.zeros((300, 400)))
    
    print("特征提取与 RANSAC 逻辑演示完毕（未实际执行以避免加载大模型耗时）。")

demo_coarse_alignment_logic()


### 4. 实验结果可视化

我们展示复现脚本 `quick_start/align2images.py` 生成的结果。

假设我们已经运行了以下命令：
```bash
python align2images.py --img1='../img/ArtMiner_Detail_Res13_10.png' --img2='../img/ArtMiner_Detail_Res13_11.png'
```
结果保存在 `output/` 目录下。


In [ ]:
def show_results(output_dir='output/'):
    try:
        # 加载生成的图片
        coarse_align = Image.open(os.path.join(output_dir, 'comb_coarse_alignment.png'))
        fine_align = Image.open(os.path.join(output_dir, 'comb_fine_alignment.png'))
        
        plt.figure(figsize=(15, 10))
        
        plt.subplot(2, 1, 1)
        plt.imshow(coarse_align)
        plt.title("Coarse Alignment Result")
        plt.axis('off')
        
        plt.subplot(2, 1, 2)
        plt.imshow(fine_align)
        plt.title("Fine Alignment Result")
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    except FileNotFoundError:
        print("未找到结果图片，请先运行 quick_start/align2images.py")

show_results()


### 5. 常见问题与踩坑指南 (Troubleshooting)

在复现过程中，由于代码库较旧（2020年），与现代 Python 环境（Python 3.13, PyTorch 2.x）存在诸多不兼容。以下是主要问题及解决方案：

#### 5.1 依赖库版本冲突
*   **问题**: `requirements.txt` 指定了旧版本的 `torch==1.7.0` 和 `torchvision==0.8.0`，这些版本在新的 CUDA 环境或 Python 3.13 下无法安装。
*   **解决**: 移除版本锁定，安装最新兼容版本。
    ```bash
    pip install torch torchvision kornia opencv-python pandas tqdm matplotlib jupyterlab
    ```

#### 5.2 `torchvision` 模型加载 API 变更
*   **问题**: 代码中使用 `models.resnet50(pretrained=True)`，但在新版 `torchvision` 中，`pretrained` 参数已被弃用，且自动下载可能因哈希校验失败而报错（`RuntimeError: invalid hash value`）。
*   **解决**:
    1.  修改代码使用 `weights` 参数：
        ```python
        # Old
        # resNetfeat = models.resnet50(pretrained=True)
        # New
        resNetfeat = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        ```
    2.  如果自动下载失败，手动下载权重文件到 `~/.cache/torch/hub/checkpoints/`。

#### 5.3 `kornia` API 变更
*   **问题**: `HomographyWarper` 类的 `warp_grid` 方法在新版 `kornia` 中已被移除或行为改变，导致 `AttributeError: 'HomographyWarper' object has no attribute 'warp_grid'`。
*   **解决**: 使用 `precompute_warp_grid` 和 `_warped_grid` 属性替代。
    ```python
    # Old
    # flowCoarse = warper.warp_grid(bestPrm)
    
    # New
    warper.precompute_warp_grid(bestPrm)
    flowCoarse = warper._warped_grid
    ```

#### 5.4 模型下载脚本
*   **问题**: `download_model.sh` 使用 Dropbox 链接，可能在某些网络环境下无法访问或速度极慢。
*   **解决**: 确保网络通畅，或者手动下载并解压到 `model/pretrained/` 目录。

